In [8]:
import tkinter as tk
import cv2
import numpy as np
import dlib
from PIL import Image, ImageTk
import time
import os 

class PongGameApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Hufana's AR Pong Game - Tkinter")
        self.root.resizable(False, False)  

        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Create canvas for video display
        self.panel = tk.Label(root)
        self.panel.pack()

        # Buttons Frame
        self.button_frame = tk.Frame(root)
        self.button_frame.pack(pady=10)

        self.restart_btn = tk.Button(self.button_frame, text="Restart", command=self.restart_game)
        self.restart_btn.pack(side="left", padx=5)

        self.screenshot_btn = tk.Button(self.button_frame, text="Screenshot", command=self.take_screenshot)
        self.screenshot_btn.pack(side="left", padx=5)

        self.quit_btn = tk.Button(self.button_frame, text="Quit", command=self.quit_game)
        self.quit_btn.pack(side="left", padx=5)

        # Load game images
        self.ball_size = 40
        self.paddle_hitbox_size = (150, 80)  # Fixed hitbox for collision
        self.paddle_visual_size = (250, 250)  # Fixed visual size for display

        # Load original images
        self.original_ball_img = cv2.imread(r"C:\Users\Phillip\Downloads\ball.png", cv2.IMREAD_UNCHANGED)
        self.original_paddle_img = cv2.imread(r"C:\Users\Phillip\Downloads\paddle.png", cv2.IMREAD_UNCHANGED)

        # Validate image paths
        if self.original_ball_img is None or self.original_paddle_img is None:
            raise FileNotFoundError("One or more image files could not be loaded.")

        # Resize only once
        self.ball_img = cv2.resize(self.original_ball_img, (self.ball_size, self.ball_size))
        self.paddle_img = cv2.resize(self.original_paddle_img, self.paddle_visual_size)

        # Initialize game variables
        self.ball_x = self.width // 2
        self.ball_y = 0
        self.ball_dx = 3
        self.ball_dy = 3
        self.game_over = False
        self.SPEED_INCREMENT = 2
        self.MAX_SPEED = 15
        self.filtered_center_x = self.width // 2
        self.filtered_center_y = self.height // 2
        self.smoothing = 0.7

        # Hit counter and high score
        self.hits = 0
        self.high_score = self.load_high_score()

        # Hardcoded HSV values for yellow
        self.lower_color = np.array([20, 100, 100])  # Yellow lower bound
        self.upper_color = np.array([30, 255, 255])  # Yellow upper bound

        # AR Setup
        self.PREDICTOR_PATH = r"C:\Users\Phillip\Downloads\shape_predictor_68_face_landmarks.dat"
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(self.PREDICTOR_PATH)
        self.glasses_img = cv2.imread(r"C:\Users\Phillip\Downloads\glasses.png", cv2.IMREAD_UNCHANGED)
        self.hat_img = cv2.imread(r"C:\Users\Phillip\Downloads\cap.png", cv2.IMREAD_UNCHANGED)

        # Store final processed frame for screenshots
        self.current_frame = None

        # Start video loop
        self.video_loop()

    def overlay_transparent(self, background, overlay, x, y):
        if overlay is None:
            return background
        bh, bw = background.shape[:2]
        h, w = overlay.shape[:2]
        if x >= bw or y >= bh or x + w <= 0 or y + h <= 0:
            return background
        if x + w > bw:
            overlay = overlay[:, :bw - x]
        if y + h > bh:
            overlay = overlay[:bh - y]
        h, w = overlay.shape[:2]
        if h <= 0 or w <= 0:
            return background
        overlay_image = overlay[..., :3].astype(float)
        mask = overlay[..., 3:] / 255.0
        roi = background[y:y+h, x:x+w].astype(float)
        if roi.shape[:2] != mask.shape[:2]:
            return background
        blended = (1.0 - mask) * roi + mask * overlay_image
        background[y:y+h, x:x+w] = blended.astype(np.uint8)
        return background

    def apply_ar_effects(self, frame, landmarks):
        points = np.array([[p.x, p.y] for p in landmarks.parts()])
        left_eye = points[36]
        right_eye = points[45]
        top_head = points[27]
        eye_width = int(np.linalg.norm(right_eye - left_eye) * 2)
        eye_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
        if self.glasses_img is not None:
            resized_glasses = cv2.resize(self.glasses_img, (eye_width, int(self.glasses_img.shape[0] * eye_width / self.glasses_img.shape[1])))
            g_x = eye_center[0] - resized_glasses.shape[1] // 2
            g_y = eye_center[1] - resized_glasses.shape[0] // 2
            frame = self.overlay_transparent(frame, resized_glasses, g_x, g_y)
        if self.hat_img is not None:
            hat_width = int(eye_width * 1.3)
            resized_hat = cv2.resize(self.hat_img, (hat_width, int(self.hat_img.shape[0] * hat_width / self.hat_img.shape[1])))
            h_x = top_head[0] - resized_hat.shape[1] // 2
            h_y = max(top_head[1] - resized_hat.shape[0] - 20, 0)
            frame = self.overlay_transparent(frame, resized_hat, h_x, h_y)
        return frame

    def load_high_score(self):
        try:
            with open("highscore.txt", "r") as f:
                return int(f.read())
        except (FileNotFoundError, ValueError):
            return 0

    def save_high_score(self):
        with open("highscore.txt", "w") as f:
            f.write(str(self.high_score))

    def video_loop(self):
        try:
            ret, frame = self.cap.read()
            if not ret:
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            Height, Width = frame.shape[:2]

            # Color Tracking
            hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_img, self.lower_color, self.upper_color)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
            contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            center_x, center_y = self.filtered_center_x, self.filtered_center_y
            if len(contours) > 0:
                c = max(contours, key=cv2.contourArea)
                M = cv2.moments(c)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    self.filtered_center_x = int(cx * self.smoothing + self.filtered_center_x * (1 - self.smoothing))
                    self.filtered_center_y = int(cy * self.smoothing + self.filtered_center_y * (1 - self.smoothing))
                    center_x, center_y = self.filtered_center_x, self.filtered_center_y

            # Overlay fixed-size paddle
            h, w = self.paddle_img.shape[:2]
            paddle_x = center_x - w // 2
            paddle_y = center_y - h // 2
            frame = self.overlay_transparent(frame, self.paddle_img, paddle_x, paddle_y)

            # AR Face Detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.detector(gray)
            for face in faces:
                landmarks = self.predictor(gray, face)
                frame = self.apply_ar_effects(frame, landmarks)

            # Pong Game Logic
            if not self.game_over:
                self.ball_x += self.ball_dx
                self.ball_y += self.ball_dy

                # Boundary collisions
                if self.ball_x - self.ball_size//2 < 0 or self.ball_x + self.ball_size//2 > Width:
                    self.ball_dx = -self.ball_dx * self.SPEED_INCREMENT
                if self.ball_y - self.ball_size//2 < 0:
                    self.ball_dy = -self.ball_dy * self.SPEED_INCREMENT
                    self.ball_y = self.ball_size//2

                # Clamp speed
                self.ball_dx = max(min(self.ball_dx, self.MAX_SPEED), -self.MAX_SPEED)
                self.ball_dy = max(min(self.ball_dy, self.MAX_SPEED), -self.MAX_SPEED)

                # Collision detection using fixed hitbox
                fixed_w, fixed_h = self.paddle_hitbox_size
                paddle_x_hitbox = center_x - fixed_w // 2
                paddle_y_hitbox = center_y - fixed_h // 2

                if (paddle_y_hitbox <= self.ball_y + self.ball_size//2 <= paddle_y_hitbox + fixed_h and 
                    paddle_x_hitbox <= self.ball_x <= paddle_x_hitbox + fixed_w):
                    self.ball_dy = -self.ball_dy * self.SPEED_INCREMENT
                    self.ball_dx = self.ball_dx * self.SPEED_INCREMENT
                    self.ball_y = paddle_y_hitbox - self.ball_size//2
                    self.hits += 1  # Increment hit counter

                # Game over
                if self.ball_y - self.ball_size//2 > Height:
                    self.game_over = True
                    if self.hits > self.high_score:
                        self.high_score = self.hits
                        self.save_high_score()

                # Draw ball
                frame = self.overlay_transparent(frame, self.ball_img, 
                                              int(self.ball_x - self.ball_size//2), 
                                              int(self.ball_y - self.ball_size//2))

                # Display score
                cv2.putText(frame, f"Hits: {self.hits}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.putText(frame, f"High Score: {self.high_score}", (10, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            else:
                cv2.putText(frame, "Game Over! Press Restart", (Width//2 - 150, Height//2), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # Save fully processed frame for screenshot
            self.current_frame = frame.copy()

            # Convert to PIL image and update GUI
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(img)
            self.panel.configure(image=img)
            self.panel.image = img

        except Exception as e:
            print(f"Error in video loop: {e}")
            self.cap.release()
            self.root.quit()

        self.root.after(10, self.video_loop)

    def restart_game(self):
        self.ball_x = self.width // 2
        self.ball_y = 0
        self.ball_dx, self.ball_dy = 3, 3
        self.game_over = False
        self.hits = 0  # Reset hit counter

    def quit_game(self):
        if self.cap.isOpened():
            self.cap.release()
        self.root.destroy()

    def take_screenshot(self):
        if self.current_frame is not None:
            save_dir = "screenshots"
            os.makedirs(save_dir, exist_ok=True)
            filename = os.path.join(save_dir, f"screenshot_{int(time.time())}.jpg")
            cv2.imwrite(filename, self.current_frame)
            print(f"Screenshot saved to: {filename}")
        else:
            print("No processed frame available for screenshot.")

if __name__ == "__main__":
    root = tk.Tk()
    app = PongGameApp(root)
    root.mainloop()

Screenshot saved to: screenshots\screenshot_1747721573.jpg
Screenshot saved to: screenshots\screenshot_1747721574.jpg
Screenshot saved to: screenshots\screenshot_1747721578.jpg
